# Tasks

Dataset collection – Download the main Steam Reviews dataset (and optionally the other ones if needed).

Tag collection – Gather the complete set of Steam tags (either from a list or by extracting tags attached to games in your dataset).

Data cleaning & preprocessing – Remove duplicates, filter uninformative reviews (“very good game”), normalize text (tokenization, lowercasing, stopword removal, etc.).

Dataset integration – Ensure reviews and tags are linked to the same games; build a consistent structure combining review text and game tags.

Exploratory data analysis – Inspect distribution of reviews per game, most frequent tags, review length, balance between positive/negative reviews, etc.

Baseline method implementation – Train Word2Vec embeddings per game and compute cosine similarity between reviews and tag embeddings.

Advanced modeling – Experiment with alternative methods (e.g., supervised learning, other embedding models, feature importance analysis).

Evaluation design – Decide on the scoring system (exact match vs. order-sensitive comparison of tags with Steam’s official tag list).

Evaluation & analysis – Run the models, compare outputs against Steam’s tags, and analyze results (per game and overall).

Reporting & discussion – Interpret findings in terms of your research questions (correlation of reviews and tags, best features for tag extraction).